In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import re
from datetime import datetime

In [2]:
#입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        page_num1 = num
        return page_num1
    elif num == 0:
        page_num2 = num+1
        return page_num2
    else:
        page_num3 = num+9*(num-1)
        return page_num3

In [3]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search,start_pg,end_pg,start_date, end_date):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + f'&sm=tab_opt&nso=so:r,p:from{start_date}to{end_date}'+"&start=" + str(start_page)
        print("생성url: ",url)
        return url
    else:
        urls= []
        for i in range(start_pg, end_pg+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + f'&sm=tab_opt&nso=so:r,p:from{start_date}to{end_date}' + "&start=" + str(page)
            urls.append(url)
        print("생성url: ",urls)
        return urls

In [4]:
def get_links (search_urls) :
  headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

  naver_urls = []
  press_name = []


  cnt = 0
  for i in search_urls :
      cnt =+ 1
      print("-----------> ", cnt)
      print("search_urls: ", i)
      response = requests.get(i, headers=headers)
      html = response.text
      soup = BeautifulSoup(html, "html.parser")
  
      news = soup.select("div > div.news_info > div.info_group > a ")     # a:nth-of-type(3)
      article = soup.select("a.info.press") # i당 10개
      articles = []
      for art in article:
        articles.append(art)

      #print("\n뉴스 : ",news)
      #print("\n아티클 : ", article)
      cnt = 0
      idx = 0
      
      for n in news:
        cnt += 1
        print("-- ", cnt, " : ", n)

        if "sid=106" in n['href'] :
              pass
        elif "sports" in n['href'] :
              pass
            
        elif  n['href'].startswith("https://n.news.naver.com"):
              print("\n네이버뉴스 입니다")
              news_url = n['href']
              naver_urls.append(news_url)
              try:
                press_name.append(articles[idx-1].text.replace("선정",""))
              except:
                print('none')
              
              
        else:
          idx += 1
          
  return naver_urls, press_name

In [5]:
def get_content (naver_urls) : 
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}


    titles = []
    contents = []
    news_date = []
    report = []

    for i in naver_urls :
        original_html = requests.get (i, headers=headers)
        h = original_html.content
        html = BeautifulSoup(h, "html.parser")
        title = html.select("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        title = ''.join(str(title)) # 문자열 합치기
        
        del_str = '<[^>]*>'
        title = re.sub(pattern=del_str, repl='',string=title)
        titles.append(title) 
        
        # 본문
        content = html.select("#dic_area")
        content = ''.join(str(content))
        content = re.sub(pattern=del_str, repl='',string=content)
        del_str2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(del_str2,'')
        # print(contents)
        contents.append(content)
        
        # 기사작성일
        p_date = html.select("span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")
        
        
        if  "sid1=106" in i:
            sports = html.select("#content > div.end_ct > div > div.article_info > span > em")
            pattern1=r'([0-9]{4})-([0-9]{2})-([0-9]{2})'
            regex_result1 = re.search(pattern1, str(sports))
            pub_date1 = regex_result1.group(0)
            news_date.append(pub_date1)
                    
        else : 
            pattern=r'([0-9]{4})-([0-9]{2})-([0-9]{2})'
            regex_result = re.search(pattern, str(p_date))
            pub_date = regex_result.group(0)
            news_date.append(pub_date)
    
        # 기자이름
        
        r = html.select("#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_journalist > a > em")
        if len(r) == 0:
            r = html.select("#contents > div.byline > p > span")
        report.append(r)
    
    reporters = []

    for a in report :
        if len(a) ==0 :
            reporters.append(r)
        
        else :
            b = a[0].text # 위치값 
            reporters.append(b.replace(' 기자',''))

    return titles, contents, news_date, reporters

In [6]:
def makeFrame (news_date, press_name, reporters, naver_urls, titles, contents):
    data = pd.DataFrame({"Pub_date" : news_date, "Press_name" : press_name, "Reporter" : reporters, "News_url" : naver_urls, "Title" : titles, "Content" : contents})
    return data

# 메인

In [8]:
#검색어 입력
search = input("검색할 키워드를 입력해주세요:")

#검색시작일
start_date = input("검색 시작 날짜 (YYYYmmdd) : ")

#검색종료일
end_date = input("검색 종료 날짜 (YYYYmmdd) : ")

#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 종료 페이지: ",page2,"페이지")   

# naver url 생성
search_urls = makeUrl(search,page, page2,start_date, end_date)
naver_urls, press_name = get_links (search_urls)
titles, contents, news_date, reporters = get_content (naver_urls)
data = makeFrame (news_date, press_name, reporters, naver_urls, titles, contents)

검색할 키워드를 입력해주세요:차박
검색 시작 날짜 (YYYYmmdd) : 20220101
검색 종료 날짜 (YYYYmmdd) : 20220131

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):3

크롤링할 종료 페이지:  3 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=차박&sm=tab_opt&nso=so:r,p:from20220101to20220131&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=차박&sm=tab_opt&nso=so:r,p:from20220101to20220131&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=차박&sm=tab_opt&nso=so:r,p:from20220101to20220131&start=21']
----------->  1
search_urls:  https://search.naver.com/search.naver?where=news&sm=tab_pge&query=차박&sm=tab_opt&nso=so:r,p:from20220101to20220131&start=1
--  1  :  <a class="info press" href="http://isplus.joins.com/" onclick="return goOtherCR(this, 'a=nws*a.prof&amp;r=1&amp;i=880002FC_000000000000000003188338&amp;g=241.0003188338&amp;u='+urlencode(this.href));" target="_blank"><span class="thumb_box"></span>일간스포츠</a>

In [9]:
data

,Pub_date,Press_name,Reporter,News_url,Title,Content
0,2022-01-31,일간스포츠,정길준,https://n.news.naver.com/mnews/article/241/000...,"[기아 레이, 연간 판매량 4만대 넘어설 듯…차박 효과]",[\n\t\t\t[일간스포츠 정길준] \n\n\n\n'더 2022 레이'. 기아...
1,2022-01-29,연합뉴스,성연재,https://n.news.naver.com/mnews/article/001/001...,"[[길따라 멋따라] ""성묘도 자제하는데"" vs ""동해안 차박이라도""]","[\n설 연휴 캠핑장 만석에 '스텔스 차박' 문의까지…""안전히 여가 즐길 방안 절실..."
2,2022-01-29,뉴시스,박주연,https://n.news.naver.com/mnews/article/003/001...,"[xEV트렌드코리아, 서울 코엑스서 3월17일부터 사흘간 개최]",[\n기사내용 요약신차 론칭·V2L 차박 특별관·에코랠리·시승 체험 등전문가·소비자...
3,2022-01-12,파이낸셜뉴스언론사,최종근,https://n.news.naver.com/mnews/article/014/000...,[캠핑·차박 열풍 타고… 대형 SUV 시장 판 커진다],"[\nGM, 초대형 ‘타호’ 사전계약 시작포드는 익스플로러 하이브리드 선봬현대차 팰..."
4,2022-01-06,노컷뉴스,포항CBS 문석준 pressmoon@cbs.co.kr,https://n.news.naver.com/mnews/article/079/000...,['차박 성지' 경주 나정고운모래해변에 '오토캠핑장' 준공],"[\n카라반 39면, 오토캠핑칸 48면 등 조성\n\n\n\n나정고운모래해변 오토캠..."
5,2022-01-20,이데일리,이윤정(yunj725@edaily.co.kr),https://n.news.naver.com/mnews/article/018/000...,"[차박여행 플랫폼 '밴플', 캠핑카 렌트 중개 베타서비스 공개]",[\n\t\t\t[이데일리 이윤정 기자] 캠핑카 차박여행 플랫폼 서비스를 운영하는 ...
6,2022-01-22,조선일보언론사,박근희 yaya@chosun.com,https://n.news.naver.com/mnews/article/023/000...,"[[아무튼, 주말] “촌스럽다고?” 2040 캠핑族 홀린 ‘90년생 갤로퍼’의 부활]",[\nMZ세대도 탄다 ‘갤로퍼 캠핑카’\t\t\t\t\t\t\t‘갤로퍼 시세가 이렇...
7,2022-01-22,머니투데이,류준영,https://n.news.naver.com/mnews/article/008/000...,[글램핑·차박에 이어 해상캠핑의 꽃 '카라반 캠핑보트' 뜨나],[\n\t\t\t[편집자주] [편집자주] 벤처·스타트업 투자흐름을 쫓아가면 미래산업...
8,2022-01-19,강원도민일보언론사,이세훈,https://n.news.naver.com/mnews/article/654/000...,[동해 망상해변 주차장 화재 발생…차박 중이던 승용차·캠핑카 등 전소],[\n\n\n\n\n동해 망상해변 주차장 화재 발생…차박 중이던 승용차·캠핑카 등 ...
9,2022-01-18,앱스토리,이미선(already@sportschosun.com),https://n.news.naver.com/mnews/article/076/000...,[[비즈브리핑] 차박하며 OTT 본다…'인포테인먼트' 강화하는 자동차업계],[\n\n\n\n\n◇볼보자동차에 '티맵 인포테인먼트' 서비스가 탑재된 모습. 사진...


In [10]:
data["Title"] = data["Title"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
data["Title"] = data["Title"].str.replace("[0-9]+", repl=r" ", regex=True)
data["Title"] = data["Title"].str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣]',repl=r" ",regex=True)

data["Content"] = data["Content"].str.replace(pat=r'(\[a-zA-Z0-9\_.+-\]+@\[a-zA-Z0-9-\]+.\[a-zA-Z0-9-.\]+)', repl=r'', regex=True)
data["Content"] = data["Content"].str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣]',repl=r" ",regex=True)
data["Content"] = data["Content"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
data["Content"] = data["Content"].str.replace(pat=r'[\r|\n]', repl=r' ', regex=True)
data["Content"] = data["Content"].str.replace(pat=r'<[^>]*>', repl=r' ', regex=True)
data["Content"] = data["Content"].str.replace("[0-9]+", repl=r" ", regex=True)

data['Reporter'] = data['Reporter'].str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣]',repl=r" ",regex=True)

In [11]:
data.isnull().sum()

Pub_date      0
Press_name    0
Reporter      0
News_url      0
Title         0
Content       0
dtype: int64

In [12]:
data = data.fillna("기타")

In [13]:
# 중복 확인 후 제거
final_df = data.copy()
dup = final_df.duplicated(['News_url']).sum()
print(dup)
final_df=final_df.drop_duplicates(['News_url'])

0


In [14]:
final_df

,Pub_date,Press_name,Reporter,News_url,Title,Content
0,2022-01-31,일간스포츠,정길준,https://n.news.naver.com/mnews/article/241/000...,기아 레이 연간 판매량 만대 넘어설 듯 차박 효과,일간스포츠 정길준 더 레이 기아 기아의...
1,2022-01-29,연합뉴스,성연재,https://n.news.naver.com/mnews/article/001/001...,길따라 멋따라 성묘도 자제하는데 동해안 차박이라도,설 연휴 캠핑장 만석에 스텔스 차박 문의까지 안전히 여가 즐길 방안 절실 ...
2,2022-01-29,뉴시스,박주연,https://n.news.naver.com/mnews/article/003/001...,트렌드코리아 서울 코엑스서 월 일부터 사흘간 개최,기사내용 요약신차 론칭 차박 특별관 에코랠리 시승 체험 등전문가 소비자 ...
3,2022-01-12,파이낸셜뉴스언론사,최종근,https://n.news.naver.com/mnews/article/014/000...,캠핑 차박 열풍 타고 대형 시장 판 커진다,초대형 타호 사전계약 시작포드는 익스플로러 하이브리드 선봬현대차 팰리...
4,2022-01-06,노컷뉴스,포항 문석준,https://n.news.naver.com/mnews/article/079/000...,차박 성지 경주 나정고운모래해변에 오토캠핑장 준공,카라반 면 오토캠핑칸 면 등 조성 나정고운모래해변 오토캠핑장 준공...
5,2022-01-20,이데일리,이윤정,https://n.news.naver.com/mnews/article/018/000...,차박여행 플랫폼 밴플 캠핑카 렌트 중개 베타서비스 공개,이데일리 이윤정 기자 캠핑카 차박여행 플랫폼 서비스를 운영하는 밴플은 ...
6,2022-01-22,조선일보언론사,박근희,https://n.news.naver.com/mnews/article/023/000...,아무튼 주말 촌스럽다고 캠핑 홀린 년생 갤로퍼 의 부활,세대도 탄다 갤로퍼 캠핑카 갤로퍼 시세가 이렇게 높은가요 ...
7,2022-01-22,머니투데이,류준영,https://n.news.naver.com/mnews/article/008/000...,글램핑 차박에 이어 해상캠핑의 꽃 카라반 캠핑보트 뜨나,편집자주 편집자주 벤처 스타트업 투자흐름을 쫓아가면 미래산업과 기업...
8,2022-01-19,강원도민일보언론사,이세훈,https://n.news.naver.com/mnews/article/654/000...,동해 망상해변 주차장 화재 발생 차박 중이던 승용차 캠핑카 등 전소,동해 망상해변 주차장 화재 발생 차박 중이던 승용차 캠핑카 등 전소 ...
9,2022-01-18,앱스토리,이미선,https://n.news.naver.com/mnews/article/076/000...,비즈브리핑 차박하며 본다 인포테인먼트 강화하는 자동차업계,볼보자동차에 티맵 인포테인먼트 서비스가 탑재된 모습 사진제공 볼보...


In [ ]:
# csv 저장
import datetime

now = datetime.datetime.now()
filename = "{}".format(search) + "_" + "{}".format(end_date)
df = final_df.to_csv(filename +'.csv', encoding = "utf-8-sig", index = False)